In [ ]:
# Install neqsim if needed (uncomment for Colab)
# !pip install neqsim

In [ ]:
from neqsim.thermo import fluid, TPflash, hydrateequilibrium
import matplotlib.pyplot as plt
import numpy as np

## 1. Introduction to Gas Hydrates

Gas hydrates are ice-like crystalline compounds where gas molecules are trapped inside water molecule cages.

**Formation conditions:**
- High pressure (typically > 10-20 bara)
- Low temperature (typically < 20-25°C)
- Presence of water and hydrate-forming gases

**Hydrate-forming gases:**
- Methane, ethane, propane, butanes
- CO2, H2S
- Nitrogen (forms hydrates at very high pressure)

In [ ]:
# Create natural gas mixture
gas = fluid("cpa")  # CPA handles water and hydrates well
gas.addComponent("nitrogen", 1.0, "mol%")
gas.addComponent("CO2", 2.0, "mol%")
gas.addComponent("methane", 85.0, "mol%")
gas.addComponent("ethane", 6.0, "mol%")
gas.addComponent("propane", 3.0, "mol%")
gas.addComponent("n-butane", 1.0, "mol%")
gas.addComponent("water", 2.0, "mol%")
gas.setMixingRule(10)  # CPA mixing rule
gas.setMultiPhaseCheck(True)

print("Natural gas mixture created")
print("Composition: N2=1%, CO2=2%, C1=85%, C2=6%, C3=3%, nC4=1%, H2O=2%")

## 2. Calculate Hydrate Formation Temperature

In [ ]:
# Calculate hydrate formation temperature at different pressures
pressures = [20, 40, 60, 80, 100, 120, 150, 200]
hydrate_temps = []

print("Hydrate Formation Temperature vs Pressure:")
print("\nPressure [bara] | Hydrate T [°C]")
print("-" * 35)

for p in pressures:
    gas.setPressure(p, "bara")
    gas.setTemperature(15.0, "C")  # Initial guess
    
    try:
        hydrate_T = hydrateequilibrium(gas)
        hydrate_temps.append(hydrate_T)
        print(f"{p:15} | {hydrate_T:.1f}")
    except Exception as e:
        hydrate_temps.append(None)
        print(f"{p:15} | Error")

In [ ]:
# Plot hydrate curve
plt.figure(figsize=(10, 6))

# Filter out None values
valid_p = [p for p, t in zip(pressures, hydrate_temps) if t is not None]
valid_t = [t for t in hydrate_temps if t is not None]

plt.plot(valid_t, valid_p, 'b-o', linewidth=2, markersize=8)

# Add regions
plt.fill_betweenx(valid_p, valid_t, -50, alpha=0.3, color='blue', label='Hydrate Zone')
plt.fill_betweenx(valid_p, valid_t, 50, alpha=0.3, color='green', label='No Hydrate')

plt.xlabel('Temperature [°C]', fontsize=12)
plt.ylabel('Pressure [bara]', fontsize=12)
plt.title('Hydrate Formation Curve - Natural Gas', fontsize=14)
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.xlim(-5, 30)
plt.ylim(0, 220)

plt.tight_layout()
plt.show()

## 3. Hydrate Risk Assessment - Subcooling

**Subcooling** is the difference between the operating temperature and the hydrate formation temperature:

$$\Delta T_{subcool} = T_{hydrate} - T_{operating}$$

- If subcooling > 0: Operating in the hydrate zone → **HIGH RISK**
- If subcooling ≤ 0: Outside hydrate zone → **Safe**

In [ ]:
# Example: Pipeline operating conditions
operating_P = 100  # bara
operating_T = 5    # °C

gas.setPressure(operating_P, "bara")
gas.setTemperature(operating_T, "C")
hydrate_T = hydrateequilibrium(gas)

subcooling = hydrate_T - operating_T

print(f"Operating conditions: P = {operating_P} bara, T = {operating_T}°C")
print(f"Hydrate formation temperature: {hydrate_T:.1f}°C")
print(f"Subcooling: {subcooling:.1f}°C")

if subcooling > 0:
    print(f"\n⚠️  WARNING: Operating {subcooling:.1f}°C inside the hydrate zone!")
    print("   Hydrate formation is thermodynamically possible.")
else:
    print(f"\n✓ Safe: Operating {-subcooling:.1f}°C outside the hydrate zone.")

## 4. Hydrate Inhibition with MEG (Monoethylene Glycol)

MEG is the most common thermodynamic hydrate inhibitor for continuous injection.

In [ ]:
# Calculate hydrate temperature depression with MEG
# We'll use different MEG concentrations in the aqueous phase

print("Effect of MEG on Hydrate Temperature at 100 bara:")
print("\nMEG [wt%] | Hydrate T [°C] | Depression [°C]")
print("-" * 50)

meg_concentrations = [0, 20, 30, 40, 50, 60]
hydrate_temps_meg = []

for meg_wt in meg_concentrations:
    # Create gas with MEG
    gas_meg = fluid("cpa")
    gas_meg.addComponent("methane", 85.0, "mol%")
    gas_meg.addComponent("ethane", 6.0, "mol%")
    gas_meg.addComponent("propane", 4.0, "mol%")
    gas_meg.addComponent("n-butane", 2.0, "mol%")
    gas_meg.addComponent("water", 3.0 * (100 - meg_wt) / 100, "mol%")
    if meg_wt > 0:
        # Approximate MEG moles based on weight fraction
        meg_moles = 3.0 * meg_wt / (100 - meg_wt) * (18.0 / 62.0)
        gas_meg.addComponent("MEG", meg_moles, "mol%")
    gas_meg.setMixingRule(10)
    gas_meg.setMultiPhaseCheck(True)
    gas_meg.setPressure(100.0, "bara")
    gas_meg.setTemperature(10.0, "C")
    
    try:
        ht = hydrateequilibrium(gas_meg)
        hydrate_temps_meg.append(ht)
        if meg_wt == 0:
            base_temp = ht
            depression = 0
        else:
            depression = base_temp - ht
        print(f"{meg_wt:9} | {ht:14.1f} | {depression:.1f}")
    except:
        hydrate_temps_meg.append(None)
        print(f"{meg_wt:9} | Error")

In [ ]:
# Plot MEG effect
valid_meg = [m for m, t in zip(meg_concentrations, hydrate_temps_meg) if t is not None]
valid_t_meg = [t for t in hydrate_temps_meg if t is not None]

plt.figure(figsize=(8, 5))
plt.plot(valid_meg, valid_t_meg, 'g-o', linewidth=2, markersize=8)
plt.xlabel('MEG Concentration [wt%]', fontsize=12)
plt.ylabel('Hydrate Formation Temperature [°C]', fontsize=12)
plt.title('Effect of MEG on Hydrate Formation Temperature\nat 100 bara', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Comparison: MEG vs Methanol

**Methanol advantages:**
- Higher depression per kg
- Lower viscosity

**MEG advantages:**
- Lower vapor losses
- Can be regenerated
- Lower toxicity

In [ ]:
# Comparison table
print("Inhibitor Comparison:")
print("\nProperty              | Methanol      | MEG")
print("-" * 55)
print("Molecular Weight      | 32 g/mol      | 62 g/mol")
print("Boiling Point         | 65°C          | 197°C")
print("Depression per wt%    | ~1.5°C        | ~1.0°C")
print("Vapor Losses          | High          | Low")
print("Regeneration          | Not practical | Standard")
print("Typical Application   | Intermittent  | Continuous")
print("")
print("Recommendation:")
print("  - MEG: Long tiebacks, continuous injection")
print("  - Methanol: Short distances, startup, emergencies")

## 6. Hydrate Prevention Strategies

In [ ]:
# Visual summary of prevention strategies
print("""
HYDRATE PREVENTION STRATEGIES
==============================

1. THERMODYNAMIC INHIBITORS (Shift equilibrium)
   ├── MEG/DEG (Glycols)
   ├── Methanol/Ethanol
   └── Salts (NaCl, CaCl2)

2. KINETIC INHIBITORS (Slow formation)
   └── KHI polymers (PVP, PVCap)
   
3. ANTI-AGGLOMERANTS (Prevent plugging)
   └── Surfactants

4. THERMAL MANAGEMENT
   ├── Insulation
   ├── Heating (electrical, hot fluid)
   └── Blowdown before cooldown

5. PRESSURE MANAGEMENT
   └── Depressurization during shutdowns

6. WATER REMOVAL
   └── Dehydration (TEG, molecular sieves)
""")

## Summary

This notebook covered:
- Gas hydrate formation conditions
- Calculating hydrate equilibrium temperature
- Understanding subcooling and hydrate risk
- Effect of thermodynamic inhibitors (MEG, methanol)
- Hydrate prevention strategies

**Key takeaways:**
- Hydrates form at high pressure and low temperature with water present
- Subcooling = T_hydrate - T_operating (positive = risk)
- MEG typically depresses hydrate temperature ~1°C per wt%
- Multiple prevention strategies can be combined